In [70]:
import pandas as pd
import sql_functions as sf
import psycopg2
import numpy as np

In [71]:
schema = 'capstone_travel_index'

In [72]:
city = sf.get_dataframe(f'SELECT * FROM {schema}.country_city_code')

In [73]:
city.head()

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
0,Tokyo,Tokyo,35.6850,139.7514,Japan,JP,JPN,Tōkyō,primary,35676000.0,1392685764
1,New York,New York,40.6943,-73.9249,United States,US,USA,New York,None,19354922.0,1840034016
2,Mexico City,Mexico City,19.4424,-99.1310,Mexico,MX,MEX,Ciudad de México,primary,19028000.0,1484247881
3,Mumbai,Mumbai,19.0170,72.8570,India,IN,IND,Mahārāshtra,admin,18978000.0,1356226629
4,São Paulo,Sao Paulo,-23.5587,-46.6250,Brazil,BR,BRA,São Paulo,admin,18845000.0,1076532519


In [74]:
city.columns

Index(['city', 'city_ascii', 'lat', 'lng', 'country', 'iso2', 'iso3',
       'admin_name', 'capital', 'population', 'id'],
      dtype='object')

In [75]:
city_lat_lud = city.drop(['city', 
                        'country', 
                        'iso2', 
                        'iso3',
                        'admin_name', 
                        'capital', 
                        'population'], axis=1)

In [76]:
city_lat_lud

,city_ascii,lat,lng,id
0,Tokyo,35.6850,139.7514,1392685764
1,New York,40.6943,-73.9249,1840034016
2,Mexico City,19.4424,-99.1310,1484247881
3,Mumbai,19.0170,72.8570,1356226629
4,Sao Paulo,-23.5587,-46.6250,1076532519
...,...,...,...,...
15488,Timmiarmiut,62.5333,-42.2167,1304206491
15489,Cheremoshna,51.3894,30.0989,1804043438
15490,Ambarchik,69.6510,162.3336,1643739159
15491,Nordvik,74.0165,111.5100,1643587468


In [77]:
frankfurt = city_lat_lud[city_lat_lud['city_ascii'] == "'s-Hertogenbosch"]

In [78]:
frankfurt

,city_ascii,lat,lng,id
2758,'s-Hertogenbosch,51.6833,5.3167,1528012333


In [79]:
def haversine(lat1, lon1, lat2, lon2):
    """
    Calculate the distance between two sets of latitude and longitude coordinates.
    """
    # Convert degrees to radians
    lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])
    
    # Radius of the Earth in kilometers
    radius = 6371
    
    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    distance = radius * c
    
    return distance

In [80]:
# Origin coordinates
origin_lat = city_lat_lud.loc[127, 'lat']
origin_lon = city_lat_lud.loc[127, 'lng']

# Calculate the distance between the origin and all other locations
for index, row in city_lat_lud.iterrows():
    distance = haversine(origin_lat, origin_lon, row['lat'], row['lng'])
    city_lat_lud.at[index, 'distance'] = distance

In [81]:
city_lat_lud[city_lat_lud['distance'] == city_lat_lud['distance'].max()]

,city_ascii,lat,lng,id,distance
5339,Waitangi,-44.0263,-176.3696,1554847194,19239.548859


## UPLOAD TO POSTGRES

In [92]:
import psycopg2

In [93]:
schema = 'capstone_travel_index'
engine = sf.get_engine()

In [94]:
# table_name = 'city_distance'

# if engine!=None:
#     try:
#         city_lat_lud.to_sql(name=table_name, # Name of SQL table
#                         con=engine, # Engine or connection
#                         if_exists='replace', # Drop the table before inserting new values 
#                         schema=schema, # Use schmea that was defined earlier
#                         index=False, # Write DataFrame index as a column
#                         chunksize=5000, # Specify the number of rows in each batch to be written at a time
#                         method='multi') # Pass multiple values in a single INSERT clause
#         print(f"The {table_name} table was imported successfully.")
#     # Error handling
#     except (Exception, psycopg2.DatabaseError) as error:
#         print(error)
#         engine = None

The city_footprint_transport table was imported successfully.
